# Notebook 1: Extract Z-scored LFP

### Overview
This Jupyter notebook focuses on extracting local field potential (LFP) traces from Spikegadgets `.rec` files, specifically for neuroscience research related to social competition trials. The notebook includes procedures for preprocessing and synchronizing raw electrophysiology data with corresponding video data and computes various metrics, including Z-scored LFPs.

### Inputs & Data Sources
- **Electrophysiology and LFP Parameters**: Constants like `EPHYS_SAMPLING_RATE`, `LFP_SAMPLING_RATE`, `TRIAL_DURATION`, etc., define basic parameters for LFP data processing.
- **Recording Information**: Stream IDs (`ECU_STREAM_ID`, `TRODES_STREAM_ID`), recording extension (`RECORDING_EXTENSION`), and paths to recording directories (`ALL_SESSION_DIR`).
- **DataFrames for Mapping and Timestamps**: `CHANNEL_MAPPING_DF` for channel mapping, and `TONE_TIMESTAMP_DF` for tone timestamps, loaded from external sources.
- **Constants for DataFrame Columns**: Names for various columns in the DataFrame, defined in an all-caps snake case format, such as `EPHYS_INDEX_COL`, `LFP_INDEX_COL`, etc.

### Output & Utility
- **Processed Data**: The notebook outputs processed data, particularly the Z-scored LFP traces, which are critical for further analysis in neuroscience research.
- **Data Files**: Outputs are saved in various formats (`CSV`, `Pickle`) in a specified output directory (`OUTPUT_DIR`).
- **Visualization**: While not explicitly mentioned, the notebook has the potential for data visualization (plots) based on processed LFP data.

### Processing Workflow
1. **LFP Extraction and Preprocessing**: 
    - Iterates through recording sessions to process `.rec` files.
    - Applies a series of preprocessing steps like bandpass filtering, notch filtering, resampling, and Z-scoring on the LFP data.
    - Exception handling for cases where the recording doesn't contain specified stream IDs.

2. **DataFrame Manipulation and Merging**:
    - Filtering `TONE_TIMESTAMP_DF` for trials with obtained LFP.
    - Addition of trial numbers and merging with `CHANNEL_MAPPING_DF`.
    - Dropping unnecessary columns and restructuring for analysis.

3. **LFP Trace Extraction for Each Trial and Brain Region**: 
    - Linking LFP calculations with trials.
    - Creating new rows for each brain region, extracting baseline, trial, and combined LFP traces.
    - Results in a comprehensive DataFrame that combines trial information with corresponding LFP traces.

4. **Data Storage**:
    - Saving processed DataFrame in both `CSV` and `Pickle` formats for easy access and future use.

### Usage Notes
- The notebook is project-specific and tailored for a particular dataset structure, requiring modifications for different data formats.
- Users should ensure file paths and directory names match their project's structure and adjust constants and parameters as needed for their specific analysis requirements.
- The notebook forms a part of a larger research framework, thus necessitating compatibility checks with other components of the project.

### Dependencies
- Python Libraries: `sys`, `os`, `glob`, `numpy`, `pandas`, `spikeinterface`
- External Data: Channel mapping and tone timestamp files, along with Spikegadgets `.rec` files.

### Customization and Scalability
- The notebook's modular design allows for easy adaptation to different datasets or extensions to include additional processing steps.
- Functions and processing steps are clearly demarcated, facilitating straightforward updates or enhancements.

### Conclusion
This notebook is a vital tool in the preprocessing and analysis of LFP data from Spikegadgets recordings, integral to neuroscience research focused on social competition trials. It offers a structured approach to handle, process, and store electrophysiological data, ensuring reproducibility and efficiency in research workflows.

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
LFP_RESAMPLE_RATIO = EPHYS_SAMPLING_RATE / LFP_SAMPLING_RATE
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [9]:
EPHYS_INDEX_COL = "time_stamp_index"
LFP_INDEX_COL = "lfp_index"
EPHYS_TIMESTAMP_COL = "time"
RECORDING_FILE_COL = "recording_file"
RECORDING_DIR_COL = "recording_dir"
BASELINE_LFP_INDEX_RANGE_COL = "baseline_lfp_index_range"
TRIAL_LFP_INDEX_RANGE_COL = "trial_lfp_index_range"
BASELINE_EPHYS_INDEX_RANGE_COL = "baseline_ephys_index_range"
TRIAL_EPHYS_INDEX_RANGE_COL = "trial_ephys_index_range"
BASELINE_VIDEOFRAME_RANGE_COL = "baseline_videoframe_range"
TRIAL_VIDEOFRAME_RANGE_COL = "trial_videoframe_range"
CURRENT_SUBJECT_COL = "current_subject"
ALL_CH_LFP_COL = "all_ch_lfp"
SUBJECT_COL = "Subject"
TRIAL_NUMBER_COL = "trial_number"
SPIKE_INTERFACE_COL = "spike_interface"
EIB_COL = "eib"

In [10]:
LFP_TRACE_COLUMNS = ["session_dir", "recording", "metadata_dir", "metadata_file", "first_dtype_name", "first_item_data", "last_dtype_name", "last_item_data", 'all_subjects', 'current_subject', 'filename']
VIDEO_COLUMNS = ['session_dir', 'recording', 'metadata_dir', 'metadata_file',
      'session_path', 'first_dtype_name', 'first_item_data',
       'all_subjects', 'current_subject', 'filename']

In [11]:
# NOTE: Change based on individual project data location

# Spreadsheet of channel mapping
CHANNEL_MAPPING_DF = pd.read_excel(os.path.join(git_root, "data/channel_mapping.xlsx"))
# Spreadsheet of tone time
SPIKEGADGETS_EXTRACTED_DF = pd.read_pickle("./proc/rce_pilot_2_00_trodes_metadata.pkl")

In [12]:
CHANNEL_MAPPING_DF.head()

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0


In [13]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
# ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_*/*.rec")
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/2023_06_*/*.rec")

In [14]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_19/20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [16]:
FULL_LFP_TRACES_PKL = "{}_full_lfp_traces_and_video_frames.pkl".format(OUTPUT_PREFIX)

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [17]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Extracting the LFP

In [18]:
SPIKEGADGETS_EXTRACTED_DF

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ..."
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ..."
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ..."
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 5541, 5541, 6927, 831...","[[1057255, 1257260], [3457285, 3657287], [5757...","[[6242, 30447], [32245, 40242], [169046, 18424..."
5,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.2,"[1.1, 1.2]",2502881,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 5541, 5541, 6927, 831...","[[1057255, 1257260], [3457285, 3657287], [5757...","[[6242, 30447], [32245, 40242], [169046, 18424..."
6,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,"[[477, 1286], [2484, 2683], [4880, 5079], [717...","[[223, 250], [252, 254], [257, 257], [258, 297...",20230621_111240_standard_comp_to_omission_D5_s...,/scrat

In [19]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)

            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)            
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero
        
    




An exception occurred: stream_id trodes is not in ['ECU']
20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged
20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged


- Combining LFP traces with metadata

In [20]:
# Filtering for rows that are the metadata for the raw recordings
# lfp_trace_condition = (SPIKEGADGETS_EXTRACTED_DF["recording"].isin(recording_name_to_all_ch_lfp) & (SPIKEGADGETS_EXTRACTED_DF["metadata_dir"] == "raw") & (SPIKEGADGETS_EXTRACTED_DF["metadata_file"] == "timestamps"))
lfp_trace_condition = (SPIKEGADGETS_EXTRACTED_DF["recording"].isin(recording_name_to_all_ch_lfp))

SPIKEGADGETS_LFP_DF = SPIKEGADGETS_EXTRACTED_DF[lfp_trace_condition].copy().reset_index(drop=True)
# Removing the columns that are not needed
# SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF[LFP_TRACE_COLUMNS]

In [21]:
# Adding the LFP traces to the metadata dataframe
SPIKEGADGETS_LFP_DF["all_ch_lfp"] = SPIKEGADGETS_LFP_DF["recording"].map(recording_name_to_all_ch_lfp)

## Get the timestamp of the LFP

In [22]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps,all_ch_lfp
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 5541, 5541, 6927, 831...","[[1057255, 1257260], [3457285, 3657287], [5757...","[[6242, 30447], [32245, 40242], [169046, 18424...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...


In [23]:
# Getting the timestamps for the LFP traces by resampling the raw timestamps
SPIKEGADGETS_LFP_DF["LFP_timestamps"] = SPIKEGADGETS_LFP_DF.apply(lambda row: np.arange(row["raw_timestamps"][0], row["raw_timestamps"][0]+(row["all_ch_lfp"].get_total_samples()*LFP_RESAMPLE_RATIO), LFP_RESAMPLE_RATIO).astype(int), axis=1)
                                                                  
                                                                  


In [24]:
SPIKEGADGETS_LFP_DF

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps,all_ch_lfp,LFP_timestamps
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 5541, 5541, 6927, 831...","[[1057255, 1257260], [3457285, 3657287], [5757...","[[6242, 30447], [32245, 40242], [169046, 18424...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
5,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_

In [25]:
SPIKEGADGETS_LFP_DF["all_ch_lfp"].iloc[0]

ZScoreRecording: 32 channels - 1.0kHz - 1 segments - 3,984,875 samples - 3,984.88s (1.11 hours) 
                 float32 dtype - 486.43 MiB

In [26]:
SPIKEGADGETS_LFP_DF["LFP_timestamps"].iloc[0][0]


0

In [27]:
SPIKEGADGETS_LFP_DF["LFP_timestamps"].iloc[0][-1]

79697480

## Adding the channel mapping

In [28]:
CHANNEL_MAPPING_DF = CHANNEL_MAPPING_DF.drop(columns=[col for col in CHANNEL_MAPPING_DF.columns if "eib" in col], errors="ignore")

In [29]:
for col in CHANNEL_MAPPING_DF.columns:
    if "spike_interface" in col:
        CHANNEL_MAPPING_DF[col] = CHANNEL_MAPPING_DF[col].fillna(0)
        CHANNEL_MAPPING_DF[col] = CHANNEL_MAPPING_DF[col].astype(int).astype(str)

In [30]:
CHANNEL_MAPPING_DF

,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,21,15,14,13,16
1,1,6.2,0,0,0,0,0
2,1,6.3,0,0,0,0,0
3,1,6.4,0,0,0,0,0
4,2,1.1,5,31,30,29,28
5,2,1.2,10,31,30,29,28
6,2,1.3,9,31,30,29,28
7,2,1.4,15,31,30,29,28


- Adding all the brain region to ch information

In [31]:
CHANNEL_MAPPING_DF[SUBJECT_COL] = CHANNEL_MAPPING_DF[SUBJECT_COL].astype(str)

- Merging the recording and the channel dataframes

In [32]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps,all_ch_lfp,LFP_timestamps
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 5541, 5541, 6927, 831...","[[1057255, 1257260], [3457285, 3657287], [5757...","[[6242, 30447], [32245, 40242], [169046, 18424...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."


In [33]:
SPIKEGADGETS_LFP_DF = pd.merge(SPIKEGADGETS_LFP_DF, CHANNEL_MAPPING_DF, left_on=CURRENT_SUBJECT_COL, right_on=SUBJECT_COL, how="left")



In [34]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,...,port_entry_timestamps,all_ch_lfp,LFP_timestamps,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,...,"[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,...,"[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.2,10,31,30,29,28
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,...,"[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,...,"[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.4,15,31,30,29,28
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,...,"[[6242, 30447], [32245, 40242], [169046, 18424...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28


# Getting the channel specific LFP traces

- Linking up all LFP calculations with all the trials

- Extracting the traces for each brain region

In [35]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,...,port_entry_timestamps,all_ch_lfp,LFP_timestamps,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,...,"[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,...,"[[102478, 104881], [110481, 147879], [193083, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.2,10,31,30,29,28
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,...,"[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,...,"[[788852, 836852], [838652, 840050], [840652, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.4,15,31,30,29,28
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,...,"[[6242, 30447], [32245, 40242], [169046, 18424...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28


In [36]:
for col in SPIKEGADGETS_LFP_DF.columns:
    if "spike_interface" in col:
        print(col)
        brain_region = col.strip(SPIKE_INTERFACE_COL).strip("_")
        trace_column = "{}_lfp_trace".format(brain_region)
        SPIKEGADGETS_LFP_DF[trace_column] = SPIKEGADGETS_LFP_DF.apply(lambda row: row[ALL_CH_LFP_COL].get_traces(channel_ids=[row[col]]).T[0], axis=1)
                                                                                                                                                        
                                                                                                                                                    

spike_interface_mPFC
spike_interface_vHPC
spike_interface_BLA
spike_interface_LH
spike_interface_MD


## Reorganizing the dataframe and removing unnecessary columns

In [37]:
SPIKEGADGETS_LFP_DF.columns

Index(['session_dir', 'original_file', 'tone_frames', 'port_entry_frames',
       'video_name', 'session_path', 'recording', 'current_subject',
       'all_subjects', 'first_timestamp', 'raw_timestamps', 'video_timestamps',
       'tone_timestamps', 'port_entry_timestamps', 'all_ch_lfp',
       'LFP_timestamps', 'Cohort', 'Subject', 'spike_interface_mPFC',
       'spike_interface_vHPC', 'spike_interface_BLA', 'spike_interface_LH',
       'spike_interface_MD', 'mPFC_lfp_trace', 'vHPC_lfp_trace',
       'BLA_lfp_trace', 'LH_lfp_trace', 'MD_lfp_trace'],
      dtype='object')

In [38]:
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF.drop(columns=[col for col in SPIKEGADGETS_LFP_DF.columns if "spike_interface" in col], errors="ignore")
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF.drop(columns=['all_ch_lfp', 'original_file', 'filename', 'current_subject'], errors="ignore")
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF.rename(columns={col: col.lower() for col in SPIKEGADGETS_LFP_DF.columns})

In [39]:
sorted_columns = sorted(SPIKEGADGETS_LFP_DF.columns
, key=lambda x: x.split("_")[-1])

In [40]:
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF[sorted_columns]

In [41]:
SPIKEGADGETS_LFP_DF.head()

,cohort,session_dir,tone_frames,port_entry_frames,video_name,session_path,recording,subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps,lfp_timestamps,mpfc_lfp_trace,vhpc_lfp_trace,bla_lfp_trace,lh_lfp_trace,md_lfp_trace
0,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.07365118, 0.11047677, 0.18994251, 0.2364590...","[-0.26256362, -0.19967411, -0.015722372, 0.150...","[0.28542638, 0.8104071, 1.2419446, 1.0261759, ...","[0.0040029064, -0.11808574, -0.052037787, 0.17...","[0.103898674, -0.011922799, 0.020439083, 0.136..."
1,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.9489666, 1.1787612, 1.587285, 1.6766496, 1....","[-0.59670866, -0.5804559, -0.500353, -0.304158...","[0.57716405, 1.2267566, 1.8559785, 1.9170899, ...","[1.0252244, 1.567814, 1.8645895, 1.6457549, 1....","[0.8103085, 0.9092951, 0.8448387, 0.6675837, 0..."
2,2,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-0.3153952, -0.28119573, -0.2697959, -0.33819...","[0.9725201, 1.5246606, 2.241502, 2.4265945, 2....","[0.18319474, 0.4846334, 0.7660871, 0.85435367,...","[0.33724487, 0.4972933, 0.52396804, 0.39059433...","[0.0032272236, -0.066158086, -0.07422615, 0.05..."
3,2,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.08225485, 0.06580388, -0.13160776, -0.78964...","[-0.75467384, -1.4102968, -2.273455, -2.297038...","[-0.14317, -0.52495664, -1.2980746, -1.0435501...","[-0.8462542, -1.1688362, -1.3489795, -1.064101...","[-0.4082438, -0.6508234, -0.85198706, -0.83423..."
4,2,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[

In [42]:
SPIKEGADGETS_LFP_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [43]:
raise ValueError("Stop here")

ValueError: Stop here